### 2025 Spring Practices in Artificial Intelligence
### College of Computer Science, Kookmin University
### Instructor : Youngwook Kim

### Assignment #1 : Implementing FixMatch algorithm



- 본 실습파일에 대한 **저작권**은 국민대학교 소프트웨어융합대학에 있습니다.
- 본 실습파일 및 솔루션을 **인터넷에 공유하는 행위는 금지**됩니다. (적발시 처벌 및 배상 책임을 질 수 있음)
- 개념 이해를 위한 **토론이나 질문은 자유롭게** 진행해도 됩니다.

In [19]:
# Step 1. 필요한 패키지들 import (수정가능)

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [20]:
# Step 2. CNN 모델 정의 및 생성 (**수정금지**)

from wideresnet import WideResNet
model = WideResNet(num_classes=10, depth=28, widen_factor=2)
model = model.cuda()

In [21]:
# Step 3. Hyperparameter (optimizer 등) 정의 (수정가능)

learning_rate = 0.0015
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
batch_size = 64
confidence_threshold = 0.95
unlabeled_weight = 1.0

In [22]:
# Step 4. CIFAR-10 training set split, dataloader 정의

# TODO: Do it yourself!
cifar10_train = torchvision.datasets.CIFAR10(root='.', train=True, download=True)

cifar10_train_data = cifar10_train.data
cifar10_train_targets = np.array(cifar10_train.targets)

print(cifar10_train_data.shape)
print(cifar10_train_targets.shape)

Files already downloaded and verified
(50000, 32, 32, 3)
(50000,)


In [23]:
# Labeled augmentation, Weak augmentation 
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Strong augmentation (RandAugment + Cutout)
strong_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandAugment(num_ops=2, magnitude=8),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0)
])

labeled_indices = np.arange(4000)
unlabeled_indices = np.arange(4000, 50000)

In [26]:
class LabeledDataset(Dataset):
    def __init__(self, data, targets, transform):
        self.data = data
        self.targets = targets
        self.transform = transform

    def __getitem__(self, idx):
        return self.transform(self.data[idx]), self.targets[idx]

    def __len__(self):
        return self.data.shape[0]

In [27]:
labeled_dataset = LabeledDataset(data=cifar10_train_data[labeled_indices],
                                 targets=cifar10_train_targets[labeled_indices],
                                 transform=transform)

labeled_loader = DataLoader(labeled_dataset, batch_size=batch_size, shuffle=True)

sample_batch = next(iter(labeled_loader))
print(sample_batch[0].shape, sample_batch[1].shape)

torch.Size([64, 3, 32, 32]) torch.Size([64])


In [28]:
class UnlabeledDataset(Dataset):
    def __init__(self, data, weak_transform, strong_transform):
        self.data = data
        self.weak_transform = weak_transform
        self.strong_transform = strong_transform

    def __getitem__(self, idx):
        img = self.data[idx]
        weak_img = self.weak_transform(img)
        strong_img = self.strong_transform(img)
        return weak_img, strong_img

    def __len__(self):
        return len(self.data)

In [29]:
unlabeled_dataset = UnlabeledDataset(data=cifar10_train_data[unlabeled_indices],
                                     weak_transform=transform,
                                     strong_transform= strong_transform)
unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=batch_size, shuffle=True)

sample_batch = next(iter(unlabeled_loader))
print(sample_batch[0].shape, sample_batch[1].shape)

torch.Size([64, 3, 32, 32]) torch.Size([64, 3, 32, 32])


In [31]:
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
test_dataset = torchvision.datasets.CIFAR10(root='.', train=False, download=True,
                                       transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

def evaluate(model, test_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.cuda(), labels.cuda()
            predicted = model(images).argmax(dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum()
    accuracy = 100 * correct / total
    return accuracy.item()

Files already downloaded and verified


In [32]:
# Step 5. CNN 모델 학습

# TODO: Do it yourself!

# 저장 여부 추적용 플래그
saved_81 = False
saved_84 = False
saved_87 = False

for epoch in range(1000):
    model.train()
    train_correct, train_total = 0, 0

    for (labeled_data, unlabeled_data) in zip(labeled_loader, unlabeled_loader):
        inputs_l, targets_l = labeled_data
        inputs_uw, inputs_us = unlabeled_data

        inputs_l, targets_l = inputs_l.cuda(), targets_l.cuda()
        inputs_uw, inputs_us = inputs_uw.cuda(), inputs_us.cuda()

        outputs_l = model(inputs_l)
        outputs_uw = model(inputs_uw)
        pseudo_targets_u = outputs_uw.argmax(dim=1).detach()

        # train accuracy 계산
        predicted_l = outputs_l.argmax(dim=1)
        train_correct += (predicted_l == targets_l).sum().item()
        train_total += targets_l.size(0)

        max_probs = torch.max(F.softmax(outputs_uw, dim=1), dim=1)[0]
        mask = max_probs.ge(confidence_threshold).float()

        outputs_us = model(inputs_us)

        supervised_loss = F.cross_entropy(outputs_l, targets_l)
        unlabeled_loss = (F.cross_entropy(outputs_us, pseudo_targets_u, reduction='none') * mask).mean()
        loss = supervised_loss + unlabeled_weight * unlabeled_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # train accuracy 출력
    train_acc = 100 * train_correct / train_total

    # test accuracy 계산
    acc = evaluate(model, test_loader)

    print(f"[Epoch {epoch+1}] Train Accuracy: {train_acc:.2f}% | Test Accuracy: {acc:.2f}%")

    # 가중치 저장
    if acc >= 87.0 and not saved_87:
        torch.save(model.state_dict(), "model_acc87.pth")
        print("✅ Saved model with accuracy ≥ 87% as model_acc87.pth")
        saved_87 = True
    elif acc >= 84.0 and not saved_84:
        torch.save(model.state_dict(), "model_acc84.pth")
        print("✅ Saved model with accuracy ≥ 84% as model_acc84.pth")
        saved_84 = True
    elif acc >= 81.0 and not saved_81:
        torch.save(model.state_dict(), "model_acc81.pth")
        print("✅ Saved model with accuracy ≥ 81% as model_acc81.pth")
        saved_81 = True

[Epoch 1] Train Accuracy: 28.40% | Test Accuracy: 10.74%
[Epoch 2] Train Accuracy: 35.77% | Test Accuracy: 20.58%
[Epoch 3] Train Accuracy: 40.05% | Test Accuracy: 22.32%
[Epoch 4] Train Accuracy: 43.35% | Test Accuracy: 17.89%
[Epoch 5] Train Accuracy: 47.62% | Test Accuracy: 32.69%
[Epoch 6] Train Accuracy: 50.70% | Test Accuracy: 20.77%
[Epoch 7] Train Accuracy: 53.98% | Test Accuracy: 34.31%
[Epoch 8] Train Accuracy: 57.00% | Test Accuracy: 35.34%
[Epoch 9] Train Accuracy: 57.62% | Test Accuracy: 28.82%
[Epoch 10] Train Accuracy: 60.10% | Test Accuracy: 29.57%
[Epoch 11] Train Accuracy: 61.42% | Test Accuracy: 24.08%
[Epoch 12] Train Accuracy: 62.52% | Test Accuracy: 37.42%
[Epoch 13] Train Accuracy: 64.50% | Test Accuracy: 35.72%
[Epoch 14] Train Accuracy: 66.05% | Test Accuracy: 33.27%
[Epoch 15] Train Accuracy: 66.45% | Test Accuracy: 40.31%
[Epoch 16] Train Accuracy: 67.58% | Test Accuracy: 33.53%
[Epoch 17] Train Accuracy: 68.47% | Test Accuracy: 38.36%
[Epoch 18] Train Accura

[Epoch 142] Train Accuracy: 97.62% | Test Accuracy: 76.12%
[Epoch 143] Train Accuracy: 97.50% | Test Accuracy: 71.97%
[Epoch 144] Train Accuracy: 97.90% | Test Accuracy: 73.70%
[Epoch 145] Train Accuracy: 97.30% | Test Accuracy: 75.70%
[Epoch 146] Train Accuracy: 98.03% | Test Accuracy: 72.94%
[Epoch 147] Train Accuracy: 97.83% | Test Accuracy: 75.87%
[Epoch 148] Train Accuracy: 97.53% | Test Accuracy: 75.85%
[Epoch 149] Train Accuracy: 97.78% | Test Accuracy: 72.69%
[Epoch 150] Train Accuracy: 98.35% | Test Accuracy: 75.68%
[Epoch 151] Train Accuracy: 97.38% | Test Accuracy: 76.15%
[Epoch 152] Train Accuracy: 97.70% | Test Accuracy: 76.69%
[Epoch 153] Train Accuracy: 97.70% | Test Accuracy: 78.43%
[Epoch 154] Train Accuracy: 98.17% | Test Accuracy: 78.05%
[Epoch 155] Train Accuracy: 98.22% | Test Accuracy: 75.01%
[Epoch 156] Train Accuracy: 98.00% | Test Accuracy: 73.53%
[Epoch 157] Train Accuracy: 98.05% | Test Accuracy: 75.13%
[Epoch 158] Train Accuracy: 97.97% | Test Accuracy: 74.6

[Epoch 280] Train Accuracy: 99.00% | Test Accuracy: 79.05%
[Epoch 281] Train Accuracy: 98.92% | Test Accuracy: 79.04%
[Epoch 282] Train Accuracy: 99.25% | Test Accuracy: 79.80%
[Epoch 283] Train Accuracy: 99.05% | Test Accuracy: 80.91%
[Epoch 284] Train Accuracy: 99.12% | Test Accuracy: 75.35%
[Epoch 285] Train Accuracy: 99.15% | Test Accuracy: 79.89%
[Epoch 286] Train Accuracy: 98.75% | Test Accuracy: 76.85%
[Epoch 287] Train Accuracy: 99.28% | Test Accuracy: 79.00%
[Epoch 288] Train Accuracy: 99.00% | Test Accuracy: 80.86%
[Epoch 289] Train Accuracy: 98.97% | Test Accuracy: 79.23%
[Epoch 290] Train Accuracy: 99.12% | Test Accuracy: 80.69%
[Epoch 291] Train Accuracy: 99.08% | Test Accuracy: 80.47%
[Epoch 292] Train Accuracy: 98.38% | Test Accuracy: 80.50%
[Epoch 293] Train Accuracy: 99.03% | Test Accuracy: 79.41%
[Epoch 294] Train Accuracy: 99.03% | Test Accuracy: 79.68%
[Epoch 295] Train Accuracy: 99.12% | Test Accuracy: 81.29%
[Epoch 296] Train Accuracy: 98.88% | Test Accuracy: 80.8

[Epoch 419] Train Accuracy: 99.22% | Test Accuracy: 82.60%
[Epoch 420] Train Accuracy: 99.22% | Test Accuracy: 81.93%
[Epoch 421] Train Accuracy: 99.33% | Test Accuracy: 78.84%
[Epoch 422] Train Accuracy: 99.45% | Test Accuracy: 82.02%
[Epoch 423] Train Accuracy: 99.75% | Test Accuracy: 81.64%
[Epoch 424] Train Accuracy: 99.80% | Test Accuracy: 81.00%
[Epoch 425] Train Accuracy: 99.22% | Test Accuracy: 79.57%
[Epoch 426] Train Accuracy: 99.38% | Test Accuracy: 82.29%
[Epoch 427] Train Accuracy: 99.40% | Test Accuracy: 80.25%
[Epoch 428] Train Accuracy: 99.50% | Test Accuracy: 82.61%
[Epoch 429] Train Accuracy: 99.60% | Test Accuracy: 80.04%
[Epoch 430] Train Accuracy: 99.10% | Test Accuracy: 82.97%
[Epoch 431] Train Accuracy: 99.22% | Test Accuracy: 81.53%
[Epoch 432] Train Accuracy: 99.40% | Test Accuracy: 80.55%
[Epoch 433] Train Accuracy: 99.15% | Test Accuracy: 82.86%
[Epoch 434] Train Accuracy: 99.38% | Test Accuracy: 83.32%
[Epoch 435] Train Accuracy: 99.42% | Test Accuracy: 82.1

[Epoch 557] Train Accuracy: 99.45% | Test Accuracy: 82.15%
[Epoch 558] Train Accuracy: 99.60% | Test Accuracy: 83.86%
[Epoch 559] Train Accuracy: 99.67% | Test Accuracy: 82.90%
[Epoch 560] Train Accuracy: 99.42% | Test Accuracy: 82.79%
[Epoch 561] Train Accuracy: 99.62% | Test Accuracy: 83.40%
[Epoch 562] Train Accuracy: 99.75% | Test Accuracy: 84.00%
[Epoch 563] Train Accuracy: 99.28% | Test Accuracy: 83.90%
[Epoch 564] Train Accuracy: 99.50% | Test Accuracy: 84.85%
[Epoch 565] Train Accuracy: 99.72% | Test Accuracy: 84.34%
[Epoch 566] Train Accuracy: 99.45% | Test Accuracy: 83.42%
[Epoch 567] Train Accuracy: 99.67% | Test Accuracy: 82.94%
[Epoch 568] Train Accuracy: 99.55% | Test Accuracy: 81.96%
[Epoch 569] Train Accuracy: 99.45% | Test Accuracy: 83.51%
[Epoch 570] Train Accuracy: 99.55% | Test Accuracy: 83.13%
[Epoch 571] Train Accuracy: 99.53% | Test Accuracy: 83.32%
[Epoch 572] Train Accuracy: 99.58% | Test Accuracy: 82.56%
[Epoch 573] Train Accuracy: 99.53% | Test Accuracy: 84.4

[Epoch 696] Train Accuracy: 99.72% | Test Accuracy: 84.86%
[Epoch 697] Train Accuracy: 99.67% | Test Accuracy: 82.33%
[Epoch 698] Train Accuracy: 99.80% | Test Accuracy: 84.94%
[Epoch 699] Train Accuracy: 99.62% | Test Accuracy: 82.26%
[Epoch 700] Train Accuracy: 99.72% | Test Accuracy: 83.22%
[Epoch 701] Train Accuracy: 99.60% | Test Accuracy: 84.85%
[Epoch 702] Train Accuracy: 99.65% | Test Accuracy: 84.13%
[Epoch 703] Train Accuracy: 99.85% | Test Accuracy: 84.21%
[Epoch 704] Train Accuracy: 99.72% | Test Accuracy: 84.48%
[Epoch 705] Train Accuracy: 99.62% | Test Accuracy: 83.52%
[Epoch 706] Train Accuracy: 99.75% | Test Accuracy: 84.22%
[Epoch 707] Train Accuracy: 99.60% | Test Accuracy: 84.37%
[Epoch 708] Train Accuracy: 99.58% | Test Accuracy: 84.32%
[Epoch 709] Train Accuracy: 99.67% | Test Accuracy: 84.74%
[Epoch 710] Train Accuracy: 99.55% | Test Accuracy: 82.70%
[Epoch 711] Train Accuracy: 99.62% | Test Accuracy: 84.35%
[Epoch 712] Train Accuracy: 99.60% | Test Accuracy: 84.3

[Epoch 835] Train Accuracy: 99.75% | Test Accuracy: 83.21%
[Epoch 836] Train Accuracy: 99.60% | Test Accuracy: 85.11%
[Epoch 837] Train Accuracy: 99.88% | Test Accuracy: 84.21%
[Epoch 838] Train Accuracy: 99.88% | Test Accuracy: 82.70%
[Epoch 839] Train Accuracy: 99.70% | Test Accuracy: 84.93%
[Epoch 840] Train Accuracy: 99.58% | Test Accuracy: 84.85%
[Epoch 841] Train Accuracy: 99.65% | Test Accuracy: 85.87%
[Epoch 842] Train Accuracy: 99.92% | Test Accuracy: 83.42%
[Epoch 843] Train Accuracy: 99.67% | Test Accuracy: 84.92%
[Epoch 844] Train Accuracy: 99.83% | Test Accuracy: 83.90%
[Epoch 845] Train Accuracy: 99.78% | Test Accuracy: 85.73%
[Epoch 846] Train Accuracy: 99.83% | Test Accuracy: 85.02%
[Epoch 847] Train Accuracy: 99.78% | Test Accuracy: 85.34%
[Epoch 848] Train Accuracy: 99.88% | Test Accuracy: 86.22%
[Epoch 849] Train Accuracy: 99.80% | Test Accuracy: 84.57%
[Epoch 850] Train Accuracy: 99.83% | Test Accuracy: 85.63%
[Epoch 851] Train Accuracy: 99.70% | Test Accuracy: 85.7

[Epoch 973] Train Accuracy: 99.80% | Test Accuracy: 83.63%
[Epoch 974] Train Accuracy: 99.83% | Test Accuracy: 85.87%
[Epoch 975] Train Accuracy: 99.90% | Test Accuracy: 86.78%
[Epoch 976] Train Accuracy: 99.83% | Test Accuracy: 86.41%
[Epoch 977] Train Accuracy: 99.92% | Test Accuracy: 86.23%
[Epoch 978] Train Accuracy: 99.90% | Test Accuracy: 86.96%
[Epoch 979] Train Accuracy: 99.72% | Test Accuracy: 86.15%
[Epoch 980] Train Accuracy: 99.85% | Test Accuracy: 86.57%
[Epoch 981] Train Accuracy: 99.78% | Test Accuracy: 86.28%
[Epoch 982] Train Accuracy: 99.80% | Test Accuracy: 85.42%
[Epoch 983] Train Accuracy: 99.75% | Test Accuracy: 86.27%
[Epoch 984] Train Accuracy: 99.92% | Test Accuracy: 85.95%
[Epoch 985] Train Accuracy: 99.80% | Test Accuracy: 86.02%
[Epoch 986] Train Accuracy: 99.75% | Test Accuracy: 85.69%
[Epoch 987] Train Accuracy: 99.83% | Test Accuracy: 85.43%
[Epoch 988] Train Accuracy: 99.67% | Test Accuracy: 85.78%
[Epoch 989] Train Accuracy: 99.97% | Test Accuracy: 86.7

In [33]:
model.load_state_dict(torch.load("model_acc87.pth"))
model = model.cuda()

In [34]:
# Step 6. CNN 모델 평가 (**수정금지**)

print(sum(p.numel() for p in model.parameters()))

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
test_dataset = torchvision.datasets.CIFAR10(root='.', train=False, download=True,
                                       transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.cuda(), labels.cuda()
        predicted = model(images).argmax(dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

print(f'Test accuracy of model on CIFAR-10: {100 * correct / total:.2f}%')

1469642
Files already downloaded and verified
Test accuracy of model on CIFAR-10: 87.22%


In [ ]:
# Step 7. 최고 성능이 나온 Hyperparameter 세팅 및 과제를 수행하면서 느낀점 및 배운점을 서술 (아래 셀에 Markdown 형식으로 작성)